International Monetary Fund API with Python
=====

## Brazil, Chile, and Colombia CPI inflation example

*January 14, 2019*<br>
*@bd_econ*

-----

I'd suggest first looking at my website's [guide](https://www.bd-econ.com/imfapi1.html) to using the IMF API. 


IMF API Documentation is [here](http://datahelp.imf.org/knowledgebase/articles/667681-json-restful-web-service)

In [8]:
# Python 3.7
import requests
import  pandas as pd
#python


#### Parameters/ Settings

In [2]:

url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
param = [('dataset', 'IFS'),
         ('freq', 'M'),
         # Brazil, Chile, Colombia
         ('country', 'BR+CL+CO'),
         ('series', 'PCPI_IX'),
         ('start', '?startPeriod=2009')]
series = '.'.join([i[1] for i in param[1:4]])

key = f'CompactData/{param[0][1]}/{series}{param[-1][1]}'

#### Make request and convert to annual percent change

In [3]:
# Combine API url with key specific to data request
r = requests.get(f'{url}{key}').json()
# data portion of results
data = r['CompactData']['DataSet']['Series']

In [4]:
i

NameError: name 'i' is not defined

In [ ]:
# Create pandas dataframe, column = country, row = obs
df = pd.DataFrame({s['@REF_AREA']: # Each country/area
                   {i['@TIME_PERIOD']: float(i['@OBS_VALUE']) 
                    for i in s['Obs']} for s in data})

# Convert index to datetime
df.index = pd.to_datetime(df.index)

# Calculate inflation rate and drop empty rows
df = (df.pct_change(12) * 100).round(1).dropna()

#### Plot results

In [ ]:
%matplotlib inline
import matplotlib
from matplotlib import pyplot as plt
matplotlib.rc('axes',edgecolor='w')

ax = df.plot(title='Inflation rate (annual percent change in CPI)', 
             color=['blue', 'red', 'green'], legend=False)
ax = plt.axhline(0, color='lightgray', lw=0.5, zorder=-1)
ax = plt.scatter([df.index[-1]] * 3, df.iloc[-1].values, 
                 color=['blue', 'red', 'green'])
for i, txt in enumerate(df.iloc[-1].values):
    ax = plt.annotate(f'  {df.iloc[-1].index[i]}: {txt:.2}%', 
                      (df.index[-1], txt-0.25))
ax = plt.annotate(f'{df.index[-1]:%b %Y}', 
                  (df.index[-1], df.iloc[-1].max() + 0.8), 
                  fontweight='bold')
ax = plt.xlim(pd.to_datetime('2009-11-01'), pd.to_datetime('2020-01-01'))
ax = plt.ylim(-1.95, 11.55)
ax = plt.yticks([10, 5, 0], ['10%', '5', '0'])

------

## Metadata and param lookup examples

#### Find available series by search term "trade"

In [ ]:
key = 'Dataflow'  # Method with series information
t = 'Trade'  # Term to find in series names
sl = requests.get(f'{url}{key}').json()['Structure']['Dataflows']['Dataflow']
# Print list of series containing text t
for s in (s for s in sl if t in s['Name']['#text']): 
    code = s['KeyFamilyRef']['KeyFamilyID']
    print(f"{s['Name']['#text']}: {code}")

#### Print the dimensions for a given series, "DOT"

In [ ]:
key = 'DataStructure/DOT'  # Method / series
dl = requests.get(f'{url}{key}').json()\
    ['Structure']['KeyFamilies']['KeyFamily']['Components']['Dimension']
for n, d in enumerate(dl):
    print(f"Dimension {n+1}: {d['@codelist']}")

#### See what values are acceptable for dimension "CL_INDICATOR_DOT"

In [ ]:
# Example: codes for dimension 3
key = f"CodeList/{dl[2]['@codelist']}"
cl = requests.get(f'{url}{key}').json()['Structure']['CodeLists']['CodeList']['Code']
for c in cl:
    print(f"{c['Description']['#text']}: {c['@value']}")

#### Bad example for how to get metadata

Probably better to just print the text, m, and read it.

In [ ]:
key = 'GenericMetadata/IFS/M.GB.PMP_IX'
m = requests.get(f'{url}{key}').json()['GenericMetadata']['MetadataSet']['AttributeValueSet']
country = m[1]['ReportedAttribute'][1]['ReportedAttribute'][3]['Value']['#text']
indicator = m[2]['ReportedAttribute'][1]['ReportedAttribute'][3]['Value']['#text']
print(f'Country: {country}; Indicator: {indicator}')

#### Another example

In [ ]:
%matplotlib inline

import matplotlib
from matplotlib import pyplot as plt
#import seaborn
matplotlib.rc('axes',edgecolor='w')

In [ ]:
import pandas as pd           # pandas version 0.18.1

# key includes two partners, B0 and W00 for EU and world
key = 'CompactData/DOT/M.GB.TMG_CIF_USD.B0+W00'

# Retrieve data from IMF API
data = requests.get(f'{url}{key}').json()

# Convert results to pandas dataframe
df = pd.DataFrame({s['@COUNTERPART_AREA'] : {pd.to_datetime(i['@TIME_PERIOD']) : 
     round(float(i['@OBS_VALUE']), 1) for i in s['Obs']} 
     for s in data['CompactData']['DataSet']['Series']})

# 12 month moving average of EU share of total
eu_share = (df['B0'].div(df['W00']) * 100).rolling(12).mean()

# Create a line plot and print most recent value as x label
title = "U.K. imports of goods: European Union share of total"
recent = f"{eu_share.index[-1].strftime('%B %Y')}: {eu_share[-1].round(1)}%"
ax = eu_share.plot(title=title)
ax = ax.set_xlabel(recent)